In [ ]:
import re
import pandas as pd

from lib import request as req
from lib import database as db

In [ ]:
def get_loto_numbers(url):
    ret = {}
    df  = req.get_dataframe(url)
    for d in df:
        times   = d.columns[1]
        times   = int(times[1:-1])
        date    = d.loc[0,:].values[1]
        numbers = d.loc[1,:].values[1:]
        bonus   = d.loc[2,:].values[1:]
        ret[times] = {
            'times'   : times,
            'date'    : date,
            'numbers' : [int(n) for n in numbers],
            'bonus'   : [int(b[1:-1]) for b in bonus[0:1]],
        }
    return ret

In [ ]:
db.delete_loto()

In [ ]:
domein = 'https://takarakuji.rakuten.co.jp'

url  = domein + "/backnumber/loto6_past/"
soup = req.get_page(url)
href = soup.find_all(href=re.compile("^/backnumber/loto6/[\d+]"))

loto = {}
for h in href:
    loto.update(get_loto_numbers(domein + h['href']))

In [ ]:
def get_loto_detail_numbers(url):
    ret = {}
    df  = req.get_dataframe(url)
    df  = df[0]
    for i in range(len(df)):
        v     = df.loc[i,:].values
        times = int(v[0][1:-1])
        ret[times] = {
            'times'   : times,
            'date'    : v[1],
            'numbers' : v[2:8],
            'bonus'   : v[8:],
        }
    return ret

href = soup.find_all(href=re.compile("^/backnumber/loto6_detail/[\d+]"))
for h in href:
    loto.update(get_loto_detail_numbers(domein + h['href']))


In [ ]:
df = pd.DataFrame(columns = [
    'times', 
    'date', 
    'one', 'two', 'three', 'four', 'five', 'six', 'seven',
    'bonus_one', 'bonus_two',
])

for k,l in loto.items():
    n = l['numbers']
    df = df.append(
        pd.Series(
            [
                l['times'],
                l['date'],
                n[0], n[1], n[2], n[3], n[4], n[5], 0,
                l['bonus'][0], 0
            ], 
            index = df.columns
        ), 
        ignore_index = True
    )

df.sort_values('times', inplace = True)
df.reset_index(drop = True, inplace = True)

db.add_loto(df)